In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('adult.csv')

In [3]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [4]:
df['income'].unique()

array(['<=50K', '>50K'], dtype=object)

In [5]:
def label_fix(label):
  if label=='<=50K':
    return 0
  else:
    return 1

In [6]:
df['income']=df['income'].apply(label_fix)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x_data=df.drop('income',axis=1)
y_labels=df['income']
x_train,x_test,y_train,y_test=train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

In [9]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income'],
      dtype='object')

In [10]:
!pip install --upgrade tensorflow==2.11.0 tensorflow-estimator==2.11.0
import tensorflow as tf

In [25]:
gender = tf.feature_column.categorical_column_with_vocabulary_list(
    'sex', ['Male', 'Female']
)
occupation = tf.feature_column.categorical_column_with_hash_bucket(
    'occupation', hash_bucket_size=1000
)
marital_status = tf.feature_column.categorical_column_with_vocabulary_list(
    'marital.status', ['Married', 'Single', 'Divorced']
)
relationship = tf.feature_column.categorical_column_with_hash_bucket(
    'relationship', hash_bucket_size=100
)
education = tf.feature_column.categorical_column_with_hash_bucket(
    'education', hash_bucket_size=100
)
workclass = tf.feature_column.categorical_column_with_hash_bucket(
    'workclass', hash_bucket_size=100
)
native_country = tf.feature_column.categorical_column_with_hash_bucket(
    'native.country', hash_bucket_size=100
)
age = tf.feature_column.numeric_column('age')
education_num = tf.feature_column.numeric_column('education.num')
capital_gain = tf.feature_column.numeric_column('capital.gain')
capital_loss = tf.feature_column.numeric_column('capital.loss')
hours_per_week = tf.feature_column.numeric_column('hours.per.week')

In [26]:
feat_cols = [
    gender,
    occupation,
    marital_status,
    relationship,
    education,
    workclass,
    native_country,
    age,
    education_num,
    capital_gain,
    capital_loss,
    hours_per_week,
]


In [21]:

# Import tensorflow_estimator directly
from tensorflow.compat.v1.estimator.inputs import pandas_input_fn
# Now you can use tfes to access the estimator module

In [28]:
input_func = pandas_input_fn(
    x=x_train, y=y_train, batch_size=100, num_epochs=None, shuffle=True
)

In [29]:
model = tf.compat.v1.estimator.LinearClassifier(feature_columns=feat_cols)


In [31]:
model.train(input_fn=input_func, steps=5000)


Instructions for updating:
Use standard file utilities to get mtimes.


In [32]:
pred_fn=pandas_input_fn(x=x_test,batch_size=len(x_test),shuffle=False)

In [33]:
  predictions=list(model.predict(input_fn=pred_fn))

In [34]:
predictions[0]

{'logits': array([-6.4510617], dtype=float32),
 'logistic': array([0.00157636], dtype=float32),
 'probabilities': array([0.9984236 , 0.00157636], dtype=float32),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'all_class_ids': array([0, 1], dtype=int32),
 'all_classes': array([b'0', b'1'], dtype=object)}

In [35]:
final_preds=[]
for pred in predictions:
  final_preds.append(pred['class_ids'][0])

In [36]:
final_preds[:10]

[0, 1, 0, 0, 0, 0, 1, 0, 0, 0]

In [37]:
from sklearn.metrics import classification_report

In [38]:
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.89      0.88      0.89      7411
           1       0.65      0.67      0.66      2358

    accuracy                           0.83      9769
   macro avg       0.77      0.78      0.77      9769
weighted avg       0.83      0.83      0.83      9769

